## Loading train and test data with transforms and loader functions



In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

data  drive  evaluate.py  main.py  model.py  __pycache__  sample_data  utils.py


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


### Plotting sample images

## The Model

In [8]:
from model import *

# Loading Custom Model
net = CIFAR10Net(norm_method='LN')

# Using cuda if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# Sending network to device
net = net.to(device)

cuda


## Model Summary



In [9]:
!pip install torchsummary
from torchsummary import summary

# Model Summary
summary(net, input_size=(3, 32, 32))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 32, 32]             108
              ReLU-2            [-1, 4, 32, 32]               0
         GroupNorm-3            [-1, 4, 32, 32]               8
           Dropout-4            [-1, 4, 32, 32]               0
            Conv2d-5            [-1, 8, 32, 32]             288
              ReLU-6            [-1, 8, 32, 32]               0
         GroupNorm-7            [-1, 8, 32, 32]              16
           Dropout-8            [-1, 8, 32, 32]               0
            Conv2d-9           [-1, 16, 32, 32]             128
             ReLU-10           [-1, 16, 32, 32]               0
        GroupNorm-11           [-1, 16, 32, 32]              32
          Dropout-12           [-1, 16, 32, 32]               0
    

## Define a Loss function and optimizer

In [10]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

## Training and Testing

In [11]:
# Load train and test model
from main import *

train = train()
test = test()

EPOCHS= 20

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    train.execute(net, device, trainloader, optimizer, criterion, epoch)
    scheduler.step()
    test.execute(net, device, testloader, criterion)

print('Finished Training')

Epoch: 0,Loss=1.34 Batch_id=390 Accuracy=36.33: 100%|██████████| 391/391 [00:18<00:00, 20.66it/s]


Test set: Average loss: 0.0115, Accuracy: 4714/10000 (47.14%)



Epoch: 1,Loss=1.23 Batch_id=390 Accuracy=51.87: 100%|██████████| 391/391 [00:19<00:00, 20.57it/s]


Test set: Average loss: 0.0094, Accuracy: 5781/10000 (57.81%)



Epoch: 2,Loss=1.21 Batch_id=390 Accuracy=61.27: 100%|██████████| 391/391 [00:17<00:00, 21.86it/s]


...

Test set: Average loss: 0.0054, Accuracy: 7695/10000 (76.95%)



Epoch: 18,Loss=0.43 Batch_id=390 Accuracy=82.21: 100%|██████████| 391/391 [00:17<00:00, 21.98it/s]


Test set: Average loss: 0.0054, Accuracy: 7697/10000 (76.97%)



Epoch: 19,Loss=0.67 Batch_id=390 Accuracy=82.22: 100%|██████████| 391/391 [00:17<00:00, 22.46it/s]


Test set: Average loss: 0.0054, Accuracy: 7704/10000 (77.04%)

Finished Training


## Evaluating Train Accuracy

In [14]:
from evaluate import *
Accuracy(net, trainloader, device, name='train')

Accuracy of the network on the  train images: 83.118 %%


## Evaluating Test Accuracy

In [15]:
Accuracy(net, testloader, device, name='test')

Accuracy of the network on the  test images: 77.04 %%
